# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless pose estimation during behavior across species*, (under revision).

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to:
- create a project
- extract training frames
- label the frames
- plot the labeled images
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video 
- plot the trajectories

*Note*: Refine a network based after the network was trained on just a few labeled images is illustrated in "Demo-labeledexample-MouseReaching.ipynb". This demo also contains an already labeled data set and is perhaps the best starting point for brand new users.

## Create a new project

It is always good idea to keep the projects seperate. This function creates a new project with subdirectories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos to the project at any stage of the project. 

In [1]:
import deeplabcut

In [2]:
task='sc_ar14_test' # Enter the name of your experiment Task
experimenter='Fassihi' # Enter the name of the experimenter
video=['D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-23-08.avi'] # Enter the paths of your videos you want to grab frames from.
# deeplabcut.create_new_project(task,experimenter,video, working_directory='Full path of the working directory',copy_videos=True/False) #change the working directory to where you want the folders created.
deeplabcut.create_new_project(task,experimenter,video, working_directory='D:\\Notebook\\sc',copy_videos=True) #change the working directory to where you want the folders created.

Created "D:\Notebook\sc\sc_ar14_test-Fassihi-2018-12-19\videos"
Created "D:\Notebook\sc\sc_ar14_test-Fassihi-2018-12-19\labeled-data"
Created "D:\Notebook\sc\sc_ar14_test-Fassihi-2018-12-19\training-datasets"
Created "D:\Notebook\sc\sc_ar14_test-Fassihi-2018-12-19\dlc-models"
Copying the videos
D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-23-08.avi
Generated "D:\Notebook\sc\sc_ar14_test-Fassihi-2018-12-19\config.yaml"

A new project with name sc_ar14_test-Fassihi-2018-12-19 is created at D:\Notebook\sc and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's needs.
 Once you have changed the configuration file, use the function 'extract_frames' to select frames for labeling.
. [OPTIONAL] Use the function 'add_new_videos' to add new videos to your project (at any stage).


'D:\\Notebook\\sc\\sc_ar14_test-Fassihi-2018-12-19\\config.yaml'

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) (algo=='uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [2]:
%matplotlib inline
path_config_file = 'D:\\Notebook\\sc\\sc_ar14_test-Fassihi-2018-12-19\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)
deeplabcut.extract_frames(path_config_file,'automatic','uniform') #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 
#deeplabcut.extract_frames(path_config_file,'automatic','uniform',crop=True, checkcropping=True) #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 

#You can change the cropping to false, then delete the checkcropping part!

Reading config file successfully...
Extracting frames based on uniform ...
Uniformly extracting of frames from 0.0  seconds to 43.73  seconds.

Frames are selected.
You can now label the frames using the function 'label_frames'.


## Label the extracted frames
Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [3]:
%gui wx
deeplabcut.label_frames(path_config_file)

Working on folder: video14-23-08


RuntimeError: wrapped C/C++ object of type DirDialog has been deleted

**Check the labels**

Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [4]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by Fassihi.
They are stored in the following folder: D:\Notebook\sc\sc_ar14_test-Fassihi-2018-12-19\labeled-data\video14-23-08_labeled.
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


If the labels need adjusted, you can use the refinement GUI to move them around! Check that out below.

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [6]:
deeplabcut.create_training_dataset(path_config_file)

D:\Notebook\sc\sc_ar14_test-Fassihi-2018-12-19\training-datasets\iteration-0\UnaugmentedDataSet_sc_ar14_testDec19  already exists!
D:\Notebook\sc\sc_ar14_test-Fassihi-2018-12-19\dlc-models\iteration-0\sc_ar14_testDec19-trainset95shuffle1  already exists!
D:\Notebook\sc\sc_ar14_test-Fassihi-2018-12-19\dlc-models\iteration-0\sc_ar14_testDec19-trainset95shuffle1//train  already exists!
D:\Notebook\sc\sc_ar14_test-Fassihi-2018-12-19\dlc-models\iteration-0\sc_ar14_testDec19-trainset95shuffle1//test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Start training - If you want to use a CPU, continue. 
### If yu want to use your GPU, you need to exit here and either work from the Docker container, your own TensorFlow installation in an Anaconda env

This function trains the network for a specific shuffle of the training dataset. 

In [7]:
deeplabcut.train_network(path_config_file)

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['nose', 'snout', 'leftEar', 'RightEar'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_sc_ar14_testDec19\\sc_ar14_test_Fassihi95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\Arash\\Anaconda3\\envs\\Myenv2\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1000,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_sc_ar14_testDec19\\Documentation_data-sc_ar14_test_95shuffle1.pi

INFO:tensorflow:Restoring parameters from C:\Users\Arash\Anaconda3\envs\Myenv2\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt


Restoring parameters from C:\Users\Arash\Anaconda3\envs\Myenv2\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt


Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'D:\\Notebook\\sc\\sc_ar14_test-Fassihi-2018-12-19\\dlc-models\\iteration-0\\sc_ar14_testDec19-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scoremap_dir': 'test', 'dataset_type': 'default', 'use_gt_segm': False, 'batch_size': 1, 'video': False, 'video_batch': False, 'crop': True, 'cropratio': 0.4, 'minsize': 100, 'leftwidth': 400, 'rightwidth': 400, 'topheight': 400, 'bottomheight': 400, 'all_joints': [[0], [1], [2], [3]], 'all_joints_names': ['nose', 'sn

iteration: 1000 loss: 0.0188 lr: 0.005
iteration: 2000 loss: 0.0099 lr: 0.005
iteration: 3000 loss: 0.0079 lr: 0.005
iteration: 4000 loss: 0.0070 lr: 0.005
iteration: 5000 loss: 0.0063 lr: 0.005
iteration: 6000 loss: 0.0059 lr: 0.005
iteration: 7000 loss: 0.0055 lr: 0.005
iteration: 8000 loss: 0.0052 lr: 0.005
iteration: 9000 loss: 0.0050 lr: 0.005
iteration: 10000 loss: 0.0049 lr: 0.005
iteration: 11000 loss: 0.0073 lr: 0.02
iteration: 12000 loss: 0.0064 lr: 0.02
iteration: 13000 loss: 0.0055 lr: 0.02
iteration: 14000 loss: 0.0052 lr: 0.02
iteration: 15000 loss: 0.0047 lr: 0.02
iteration: 16000 loss: 0.0045 lr: 0.02
iteration: 17000 loss: 0.0043 lr: 0.02
iteration: 18000 loss: 0.0039 lr: 0.02
iteration: 19000 loss: 0.0037 lr: 0.02
iteration: 20000 loss: 0.0038 lr: 0.02
iteration: 21000 loss: 0.0037 lr: 0.02
iteration: 22000 loss: 0.0035 lr: 0.02
iteration: 23000 loss: 0.0037 lr: 0.02
iteration: 24000 loss: 0.0034 lr: 0.02
iteration: 25000 loss: 0.0033 lr: 0.02
iteration: 26000 loss: 0

KeyboardInterrupt: 

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
#deeplabcut.evaluate_network(path_config_file)

deeplabcut.evaluate_network(path_config_file,Shuffles=[1], plotting=True)

## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [4]:
#path_config_file = 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-15\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

#videofile_path = ['D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-23-50.avi'] #Enter the list of videos to analyze.
videofile_path = ['D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-24-01.avi','D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-23-50'] #Enter the list of videos to analyze.
path_config_file = 'D:\\Notebook\\sc\\sc_ar14_test-Fassihi-2018-12-19\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)
import os
Mainfolder = 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos'
f = os.listdir(Mainfolder)
text_files = [f for f in os.listdir(Mainfolder) if f.endswith('.avi')]


my_list = text_files
thisApen = Mainfolder+'kir'
thisApen = thisApen.replace("kir", "\\")
string = thisApen
my_new_list = [ string + x for x in my_list]
print (my_new_list)
#my_new_list = ['D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-28-31.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-28-41.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-28-52.avi']

#deeplabcut.analyze_videos(path_config_file,videofile_path)
# my_new_list was created in prevouse cell as all videos in the folder 
deeplabcut.analyze_videos(path_config_file,my_new_list,shuffle=1, save_as_csv=True)


['D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-22-57.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-23-08.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-23-19.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-23-26.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-23-37.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-23-50.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-24-01.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-24-08.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-24-21.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-24-32.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-24-43.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-24-54.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-25-05.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-25-16.avi', 'D:\\Notebook\\sc\\

INFO:tensorflow:Restoring parameters from D:\Notebook\sc\sc_ar14_test-Fassihi-2018-12-19\dlc-models\iteration-0\sc_ar14_testDec19-trainset95shuffle1\train\snapshot-100000


D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-22-57.avi
Starting %  D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-22-57.avi
Loading  D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-22-57.avi
Duration of video [s]:  32.6 , recorded with  15.0 fps!
Overall # of frames:  489 without cropped frame dimensions:  1200 680
Starting to extract posture





  0%|                                                                                          | 0/489 [00:00<?, ?it/s]


  4%|███▎                                                                             | 20/489 [00:03<01:28,  5.29it/s]


  6%|████▉                                                                            | 30/489 [00:07<01:57,  3.92it/s]


  8%|██████▋                                                                          | 40/489 [00:10<01:57,  3.83it/s]


 10%|████████▎                                                                        | 50/489 [00:14<02:12,  3.31it/s]


 12%|█████████▉                                                                       | 60/489 [00:17<02:05,  3.42it/s]


 14%|███████████▌                                                                     | 70/489 [00:21<02:15,  3.09it/s]


 16%|█████████████▎                                                                   | 80/489 [00:24<02:06,  3.24it/s]


 18%|██████████████▉ 

Detected frames:  489
Saving results in D:\Notebook\sc\sc-Fassihi-2018-12-16\videos...
Saving csv poses!
D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-23-08.avi
Starting %  D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-23-08.avi
Loading  D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-23-08.avi
Duration of video [s]:  43.73 , recorded with  15.0 fps!
Overall # of frames:  656 without cropped frame dimensions:  1200 680
Starting to extract posture


...

Detected frames:  271
Saving results in D:\Notebook\sc\sc-Fassihi-2018-12-16\videos...
Saving csv poses!
D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-28-10.avi
Starting %  D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-28-10.avi
Loading  D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-28-10.avi
Duration of video [s]:  23.93 , recorded with  15.0 fps!
Overall # of frames:  359 without cropped frame dimensions:  1200 680
Starting to extract posture



































  0%|                                                                                          | 0/359 [00:00<?, ?it/s]
































  6%|████▌                                                                            | 20/359 [00:03<00:51,  6.61it/s]
































  8%|██████▊                                                                          | 30/359 [00:07<01:17,  4.25it/s]
































 11%|█████████                                                                        | 40/359 [00:10<01:19,  4.03it/s]
































 14%|███████████▎                                                                     | 50/359 [00:14<01:31,  3.37it/s]
































 17%|█████████████▌                                                                   | 60/359 [00:16<01:26,  3.44it/s]
































 19%|███████████████▊                                  

Detected frames:  359
Saving results in D:\Notebook\sc\sc-Fassihi-2018-12-16\videos...
Saving csv poses!
D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-28-17.avi
Starting %  D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-28-17.avi
Loading  D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-28-17.avi
Duration of video [s]:  51.27 , recorded with  15.0 fps!
Overall # of frames:  769 without cropped frame dimensions:  1200 680
Starting to extract posture




































  0%|                                                                                          | 0/769 [00:00<?, ?it/s]

































  3%|██                                                                               | 20/769 [00:02<01:45,  7.07it/s]

































  4%|███▏                                                                             | 30/769 [00:06<02:41,  4.56it/s]

































  5%|████▏                                                                            | 40/769 [00:09<02:51,  4.24it/s]

































  7%|█████▎                                                                           | 50/769 [00:13<03:27,  3.47it/s]

































  8%|██████▎                                                                          | 60/769 [00:16<03:21,  3.51it/s]

































  9%|███████▎                                   

Detected frames:  769
Saving results in D:\Notebook\sc\sc-Fassihi-2018-12-16\videos...
Saving csv poses!
D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-28-31.avi
Starting %  D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-28-31.avi
Video already analyzed! D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-28-31DeepCut_resnet50_sc_ar14_testDec19shuffle1_100000.h5
D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-28-41.avi
Starting %  D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-28-41.avi
Video already analyzed! D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-28-41DeepCut_resnet50_sc_ar14_testDec19shuffle1_100000.h5
D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-28-52.avi
Starting %  D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-28-52.avi
Video already analyzed! D:\Notebook\sc\sc-Fassihi-2018-12-16\videos\video14-28-52DeepCut_resnet50_sc_ar14_testDec19shuffle1_100000.h5
The videos are analyzed. Now your research can truly start! 
 You can create labeled 

## Extract outlier frames [optional step]
This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [5]:
deeplabcut.extract_outlier_frames(path_config_file,['D:\\Notebook\\sc\\sc-Fassihi-2018-12-15\\videos\\video14-23-37.avi])

SyntaxError: EOL while scanning string literal (<ipython-input-5-591c7f34e1a7>, line 1)

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset.

In [8]:
%gui wx
deeplabcut.refine_labels(path_config_file)

ERROR:root:Cannot activate multiple GUI eventloops


In [0]:
#Once all folders are relabeled, check them and advance. See how to check labels, above!
deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [11]:
deeplabcut.create_training_dataset(path_config_file)

D:\Notebook\sc\sc-Fassihi-2018-12-16\training-datasets\iteration-0\UnaugmentedDataSet_scDec16  already exists!
D:\Notebook\sc\sc-Fassihi-2018-12-16\dlc-models\iteration-0\scDec16-trainset95shuffle1  already exists!
D:\Notebook\sc\sc-Fassihi-2018-12-16\dlc-models\iteration-0\scDec16-trainset95shuffle1//train  already exists!
D:\Notebook\sc\sc-Fassihi-2018-12-16\dlc-models\iteration-0\scDec16-trainset95shuffle1//test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


In [56]:
import os
Mainfolder = 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos'
f = os.listdir(Mainfolder)
text_files = [f for f in os.listdir(Mainfolder) if f.endswith('.avi')]


my_list = text_files
thisApen = Mainfolder+'kir'
thisApen = thisApen.replace("kir", "\\")
string = thisApen
my_new_list = [ string + x for x in my_list]
print (my_new_list)



['D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-22-57.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-23-08.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-23-19.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-23-26.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-23-37.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-23-50.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-24-01.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-24-08.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-24-21.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-24-32.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-24-43.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-24-54.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-25-05.avi', 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-25-16.avi', 'D:\\Notebook\\sc\\

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [54]:
deeplabcut.create_labeled_video(path_config_file,my_new_list,save_frames) # my_new_list was created in prevouse cell as all videos in the folder 

NameError: name 'save_frames' is not defined

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [4]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)



UsageError: unrecognized arguments: #for making interactive plots.


In [53]:
path_config_file = 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\config.yaml'
path_config_file = 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

videofile_path = 'D:\\Notebook\\sc\\sc-Fassihi-2018-12-16\\videos\\video14-24-01.avi'
#print(path_config_file)


IndentationError: expected an indented block (<ipython-input-44-9ddf1c8e15a7>, line 9)